Cluster, plot & visualize CLIP embeddings for gaze tiles
(AJH Apr 2025)


In [1]:
# Import libraries
import pandas as pd
import io
import numpy as np
import torch
import os
import skimage
import IPython.display
import matplotlib.pyplot as plt
from PIL import Image
from sklearn.cluster import KMeans
from sklearn.metrics import normalized_mutual_info_score


In [2]:
# Load embedding file
from google.colab import files

uploaded = files.upload()


Saving embeddings_zscore.csv to embeddings_zscore.csv


In [3]:
# data frame
for fn in uploaded.keys():
  df = pd.read_csv(io.BytesIO(uploaded[fn]))


In [ ]:
df

,row_id,0,1,2,3,4,5,6,7,8,...,502,503,504,505,506,507,508,509,510,511
0,tulip019_shoehorn_block1_fix22.jpeg,0.002747,0.08496,-0.11554,-0.26420,0.12354,-0.10420,0.25880,1.3700,0.1483,...,-0.30520,0.02147,0.36060,-0.2305,0.15880,-0.205300,-0.35700,0.7627,-0.02715,0.036300
1,tulip023_rolodex_block2_fix31.jpeg,-0.125100,-0.10760,0.02803,-0.06305,0.25070,0.46000,-0.09100,0.6123,-0.3800,...,-0.18190,-0.45480,0.24260,-0.2659,0.25800,-0.205300,-0.07794,0.6550,0.22200,0.155500
2,tulip019_floppydisk_block1_fix18.jpeg,0.242400,-0.15420,0.02025,-0.16590,-0.04718,-0.12260,0.10706,0.6520,-0.0787,...,0.04593,-0.19260,0.27950,-0.3213,0.35820,-0.303200,-0.53100,0.6190,0.13830,-0.174400
3,tulip022_bulbplanter_block1_fix17.jpeg,0.065900,0.28560,0.15610,-0.15880,0.14770,-0.18660,0.31450,0.6543,-0.2477,...,0.18070,-0.19640,0.10596,-0.3716,-0.05234,-0.055730,-0.23930,0.6130,0.23580,0.007217
4,tulip012_bloodpressurecuff_block1_fix92.jpeg,-0.069950,0.10120,0.20150,-0.13550,-0.15750,0.14010,0.29540,0.7417,-0.2109,...,-0.08514,-0.28120,0.39100,-0.5063,0.41580,-0.212200,-0.33890,0.6770,0.25930,-0.007965
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19028,tulip020_handmixer_block1_fix37.jpeg,0.008804,0.08563,0.21040,-0.35230,-0.10754,-0.18140,0.07733,0.5083,-0.4100,...,-0.11115,-0.18210,0.07070,-0.2395,-0.06570,-0.278600,-0.15900,0.3594,-0.13130,0.285400
19029,tulip013_rolodex_block1_fix29.jpeg,0.193600,0.03073,0.09740,-0.19980,0.05884,0.02089,0.07635,0.8223,-0.3384,...,0.03467,-0.16160,-0.04678,-0.2013,0.29030,-0.094060,-0.39820,0.4250,0.19350,0.095700
19030,tulip019_stethoscope_block1_fix5.jpeg,-0.266600,0.19930,0.15530,-0.42330,0.02010,0.16860,-0.05725,0.2825,-0.4731,...,0.09480,-0.37000,0.09700,-0.0803,0.02753,-0.235200,0.12230,0.7510,-0.28740,0.118800
19031,tulip014_shoehorn_block1_fix26.jpeg,0.086850,0.10016,0.30080,-0.29370,-0.15560,-0.03845,0.12960,0.5780,-0.3270,...,0.25950,-0.31200,-0.10547,-0.3345,0.33810,-0.221100,-0.18690,0.5356,0.40400,0.128900


In [4]:

# Add variables using file name
df['subjectID'] = df['row_id'].str.split('_').str[0] # the first chain
#groupedBySub = df.groupby('subjectID')

df['objectID'] = df['row_id'].str.split('_').str[1] # the second chain (e.g., "shoehorn")
#groupedBySub = df.groupby('objectID')

df['block'] = df['row_id'].str.split('_').str[2].str[5] # the third chain (e.g., "1 vs 2")#groupedBySub = df.groupby('block')

df['fixNum'] = df['row_id'].str.split('_').str[3].str.split('.').str[0].str[3:] # the last chain
#groupedBySub = df.groupby('fixNum')



In [6]:
df

,row_id,0,1,2,3,4,5,6,7,8,...,506,507,508,509,510,511,subjectID,objectID,block,fixNum
0,tulip019_shoehorn_block1_fix22.jpeg,-0.1973,-0.66900,-1.5310,-0.59500,-0.04514,-1.1670,1.01000,2.15600,1.4020,...,-0.1852,-1.3040,-0.4092,0.0823,-1.1800,-0.6035,tulip019,shoehorn,1,22
1,tulip023_rolodex_block2_fix31.jpeg,-1.1200,-1.83200,-0.4373,0.76860,0.78000,3.0740,-1.35100,-0.03818,-0.8857,...,0.4421,-1.3040,1.1980,-0.2063,0.5635,0.2098,tulip023,rolodex,2,31
2,tulip019_floppydisk_block1_fix18.jpeg,1.5330,-2.11300,-0.4963,0.07117,-1.15200,-1.3050,-0.01441,0.07635,0.4192,...,1.0750,-2.0570,-1.4110,-0.3015,-0.0222,-2.0410,tulip019,floppydisk,1,18
3,tulip022_bulbplanter_block1_fix17.jpeg,0.2585,0.54350,0.5390,0.11920,0.11163,-1.7860,1.38600,0.08344,-0.3130,...,-1.5205,-0.1528,0.2686,-0.3186,0.6600,-0.8020,tulip022,bulbplanter,1,17
4,tulip012_bloodpressurecuff_block1_fix92.jpeg,-0.7220,-0.57100,0.8853,0.27730,-1.86800,0.6700,1.25700,0.33670,-0.1539,...,1.4390,-1.3550,-0.3052,-0.1475,0.8240,-0.9053,tulip012,bloodpressurecuff,1,92
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19028,tulip020_handmixer_block1_fix37.jpeg,-0.1536,-0.66500,0.9526,-1.19300,-1.54400,-1.7470,-0.21500,-0.33940,-1.0160,...,-1.6045,-1.8660,0.7305,-0.9960,-1.9090,1.0960,tulip020,handmixer,1,37
19029,tulip013_rolodex_block1_fix29.jpeg,1.1810,-0.99600,0.0916,-0.15890,-0.46480,-0.2264,-0.22160,0.57000,-0.7060,...,0.6460,-0.4478,-0.6470,-0.8203,0.3638,-0.1981,tulip013,rolodex,1,29
19030,tulip019_stethoscope_block1_fix5.jpeg,-2.1400,0.02213,0.5327,-1.67500,-0.71630,0.8833,-1.12400,-0.99370,-1.2900,...,-1.0160,-1.5330,2.3520,0.0509,-3.0000,-0.0408,tulip019,stethoscope,1,5
19031,tulip014_shoehorn_block1_fix26.jpeg,0.4097,-0.57700,1.6410,-0.79540,-1.85500,-0.6724,0.13800,-0.13720,-0.6562,...,0.9480,-1.4250,0.5703,-0.5250,1.8380,0.0283,tulip014,shoehorn,1,26


In [5]:
# prompt: save the new dataframe to my computer as a csv

from google.colab import files
df.to_csv('updated_dataframe.csv', encoding = 'utf-8-sig')
files.download('updated_dataframe.csv')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# Create a new column 'embedding_vector' containing vectors of the first 512 columns
df['embedding_vector'] = df.iloc[:, 1:513].values.tolist()
#df


In [ ]:
# Remove columns 1 through 512 from the DataFrame
df = df.drop(df.columns[1:513], axis=1)

In [ ]:
# Initialize a list to store NMI scores for each subject
nmi_scores = []

In [ ]:
# Across all particiants, how well can we cluster embeddings by the objectID?

k = 12

# get the embeddings for their fixations
tileEmbeddings = np.array(df['embedding_vector'].tolist())

# Create a KMeans object
kmeans = KMeans(n_clusters=k, random_state=0)

# Fit the KMeans model to your image features
kmeans.fit(tileEmbeddings)

# Get cluster labels for each image
df['cluster_labels'] = kmeans.labels_
#print(subject_df)

# Calculate NMI for the current subject
nmi = normalized_mutual_info_score(df['cluster_labels'], df['objectID'])

In [ ]:
nmi

In [ ]:
df

,row_id,subjectID,objectID,block,fixNum,embedding_vector,cluster_labels
0,tulip019_shoehorn_block1_fix22.jpeg,tulip019,shoehorn,1,22,"[0.0003135204315185, 0.0065879821777343, -0.00...",6
1,tulip023_rolodex_block2_fix31.jpeg,tulip023,rolodex,2,31,"[-0.0104293823242187, -0.0090484619140625, 0.0...",3
2,tulip019_floppydisk_block1_fix18.jpeg,tulip019,floppydisk,1,18,"[0.019927978515625, -0.0126495361328125, 0.001...",9
3,tulip022_bulbplanter_block1_fix17.jpeg,tulip022,bulbplanter,1,17,"[0.0055084228515625, 0.02386474609375, 0.01308...",9
4,tulip012_bloodpressurecuff_block1_fix92.jpeg,tulip012,bloodpressurecuff,1,92,"[-0.0056571960449218, 0.0080337524414062, 0.01...",4
...,...,...,...,...,...,...,...
19028,tulip020_handmixer_block1_fix37.jpeg,tulip020,handmixer,1,37,"[0.0007333755493164, 0.0070075988769531, 0.017...",7
19029,tulip013_rolodex_block1_fix29.jpeg,tulip013,rolodex,1,29,"[0.0160064697265625, 0.00244140625, 0.00811767...",0
19030,tulip019_stethoscope_block1_fix5.jpeg,tulip019,stethoscope,1,5,"[-0.0228424072265625, 0.016937255859375, 0.013...",3
19031,tulip014_shoehorn_block1_fix26.jpeg,tulip014,shoehorn,1,26,"[0.00726318359375, 0.0082855224609375, 0.02481...",4


In [ ]:
# prompt: plot a confusion matrix

import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import seaborn as sns

# Assuming 'df' and 'objectID' and 'cluster_labels' are defined as in the previous code
cm = confusion_matrix(df['objectID'], df['cluster_labels'])

plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.xlabel('Predicted labels')
plt.ylabel('True labels')
plt.title('Confusion Matrix')
plt.show()


ValueError: Mix of label input types (string and number)

In [ ]:
# Loop through subjects
for subject in df['subjectID'].unique():
  # Get subject name
  subject_name = subject

  # Filter the DataFrame for the current subject
  subject_df = df[df['subjectID'] == subject]

  # Count unique object IDs for the current subject
  trueCount = subject_df['objectID'].nunique()

  # Choose the number of clusters (k)
  k = trueCount

  # get the embeddings for their fixations
  tileEmbeddings = np.array(subject_df['embedding_vector'].tolist())
  #print(tileEmbeddings)

  # Create a KMeans object
  kmeans = KMeans(n_clusters=k, random_state=0)

  # Fit the KMeans model to your image features
  kmeans.fit(tileEmbeddings)

  # Get cluster labels for each image
  subject_df['cluster_labels'] = kmeans.labels_
  #print(subject_df)

  # Calculate NMI for the current subject
  nmi = normalized_mutual_info_score(subject_df['cluster_labels'], subject_df['objectID'])
  nmi_scores.append(nmi)



In [ ]:
# Calculate the average NMI across all subjects
average_nmi = np.mean(nmi_scores)
print(nmi_scores)
print(average_nmi)

[np.float64(0.13383960986457544), np.float64(0.21001952879959235), np.float64(0.38623607766419377), np.float64(0.245828430489196), np.float64(0.23446995686291303), np.float64(0.11009165646956212), np.float64(0.20595120108172127), np.float64(0.23739112415925973), np.float64(0.08540821149869805), np.float64(0.12299810522945268), np.float64(0.2436574594511323), np.float64(0.1953041255145471), np.float64(0.3441376058057028), np.float64(0.33234241185585134), np.float64(0.2860650800052816), np.float64(0.18451279836730958), np.float64(0.3544802224721015), np.float64(0.3012849996684256), np.float64(0.6688068610177016), np.float64(0.4769541363812494), np.float64(0.2814099015632248), np.float64(0.20850365364921314)]
0.2658951435395866
